In [15]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

from keras.utils.visualize_util import plot
from IPython.display import Image, display

from CMS_SURF_2016.utils.preprocessing import ObjectProfile, \
        preprocessFromPandas_label_dir_pairs, label_dir_pairs_args_decoder, \
        genFrom_label_dir_pairs,maxMutualLength, start_num_fromSplits
from CMS_SURF_2016.utils.metrics import plot_history, print_accuracy_m
from CMS_SURF_2016.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.layers.lorentz import Lorentz, _lorentz
from CMS_SURF_2016.layers.slice import Slice

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.utils.visualize_util import plot
from keras.layers.advanced_activations import LeakyReLU


dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')




using gpu0
using theano


In [16]:
#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'Charge', "PT_ET", "Eta", "Phi", "Dxy_Ehad_Eem"]
vecsize = len(observ_types)
epochs = 100
batch_size = 100
sample_start = 0
num_samples = 10000


object_profiles = [ObjectProfile("Electron",5),
                    ObjectProfile("MuonTight", 5),
                    ObjectProfile("Photon", 25),
                    ObjectProfile("MissingET", 1),
                    ObjectProfile("EFlowPhoton",100, sort_columns=["PT_ET"], sort_ascending=False),  #1300
                    ObjectProfile("EFlowNeutralHadron",100, sort_columns=["PT_ET"], sort_ascending=False),  #1000
                    ObjectProfile("EFlowTrack",100, sort_columns=["PT_ET"], sort_ascending=False)]  #1050


label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_unjoined/")
            ]
#ttbar_files = glob.glob("/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/*.h5")
#WJet_files = glob.glob("/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/*.h5")
#qcd_files = glob.glob("/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_unjoined/*.h5")

#files = {0:ttbar_files, 1:WJet_files}


In [17]:
archive_dir = "/data/shared/Delphes/keras_archive/"
#def getXYGenerator()
SNs = start_num_fromSplits((.7,.15,.15), maxMutualLength(label_dir_pairs, object_profiles))
print(SNs)
gen_lambda =  lambda s : (DataProcedure(archive_dir,
                                           False,
                                        genFrom_label_dir_pairs,
                                       start=s[0],
                                       samples_per_label=s[1],
                                       stride=10000,
                                        batch_size=100,
                                       archive_dir=archive_dir,
                                       label_dir_pairs=label_dir_pairs[:2],
                                       object_profiles=object_profiles,
                                       observ_types=observ_types),
                         len(label_dir_pairs)*s[1])
l = [gen_lambda(s) for s in SNs]
train, num_train = l[0]
val,   num_val   = l[1]
test,  num_test  = l[2]

gen = train.getData()

[(0, 88816), (88816, 19032), (107848, 19032)]
Generating DataProcedure in range(0,88816):
   From 2 labels in range(0,10000) for 2x10000 = 20000 Samples
   From 2 labels in range(10000,20000) for 2x10000 = 20000 Samples
   From 2 labels in range(20000,30000) for 2x10000 = 20000 Samples
   From 2 labels in range(30000,40000) for 2x10000 = 20000 Samples
   From 2 labels in range(40000,50000) for 2x10000 = 20000 Samples
   From 2 labels in range(50000,60000) for 2x10000 = 20000 Samples
   From 2 labels in range(60000,70000) for 2x10000 = 20000 Samples
   From 2 labels in range(70000,80000) for 2x10000 = 20000 Samples
   From 2 labels in range(80000,88816) for 2x8816 = 17632 Samples


In [18]:
#X_train, y_train = preprocessFromPandas_label_dir_pairs(label_dir_pairs, sample_start, num_samples, object_profiles,observ_types)

In [19]:
#print(len(X_train["Electron"]))
#X_train

        

In [ ]:
def genModel(name,out_dim, depth, width, dense_activation="relu", dropout = 0.0,sphereCoords=True):
	inputs = []
	mergelist = []
	for i, profile in enumerate(object_profiles):
		# print(o)
		inp = a = Input(shape=(profile.max_size, vecsize), name="input_"+str(i))
		inputs.append(inp)

		if(name == 'lorentz'):
			b1 = Lorentz(sphereCoords=sphereCoords, name="lorentz_"+str(i))(a)
		else:
			b1 = Slice('[:,0:4]',name='slice_1_'+str(i))(a)
		b1 = Flatten(name="flatten1_"+str(i))(b1)

		
		b2 = Slice('[:,4:9]',name='slice_2_'+str(i))(a)
		b2 = Flatten(name="flatten_2_"+str(i))(b2)
		# b2 = Dense(10, activation='relu')(b2)
		mergelist.append(b1)
		mergelist.append(b2)
	# print(mergelist)
	a = merge(mergelist,mode='concat', name="merge")
	# a = Flatten()(a)
	for i in range(depth):
		a =  Dense(width, activation=dense_activation, name="dense_"+str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
	dense_out = Dense(out_dim, activation='sigmoid', name='main_output')(a)
	model = Model(input=inputs, output=dense_out, name=name)
	return model

In [ ]:
archive_dir = "/data/shared/Delphes/keras_archive/"
earlyStopping = EarlyStopping(verbose=1, patience=10)
overfitStopping = OverfitStopping(verbose=1, patience=20)
trials = []
ldps_pairs = []
for i, j in [(0,1),(1,2),(2,0)]:
    ldps = [label_dir_pairs[i]] + [label_dir_pairs[j]]
    SNs = start_num_fromSplits((.7,.15,.15), maxMutualLength(label_dir_pairs, object_profiles))
    print(SNs)
    gen_lambda =  lambda s : (DataProcedure(archive_dir,
                                           False,
                                           genFrom_label_dir_pairs,
                                           start=s[0],
                                           samples_per_label=s[1],
                                           stride=10000,
                                            batch_size=100,
                                           archive_dir=archive_dir,
                                           label_dir_pairs=ldps,
                                           object_profiles=object_profiles,
                                           observ_types=observ_types),
                             len(ldps)*s[1])
    l = [gen_lambda(s) for s in SNs]
    train, num_train = l[0]
    val,   num_val   = l[1]
    test,  num_test  = l[2]
    
    labels = [x[0] for x in ldps]
    for name in ['lorentz', 'not_lorentz']:
        for sphereCoords in [True,False]:
            for depth in [1,2]:
                for width in [10,100]:
                    for activation in ['relu']:
                        for dropout in [0.0]:
                            activation_name = activation if isinstance(activation, str) \
                                                else activation.__name__

                            model = genModel(name, len(ldps), depth, width, activation, dropout, sphereCoords)

                            trial = KerasTrial(archive_dir, name=name, model=model)

                            trial.setTrain(train_procedure=train,
                                           samples_per_epoch=num_train
                                          )
                            trial.setValidation(val_procedure=val,
                                               nb_val_samples=num_val)
                            trial.setCompilation(loss='binary_crossentropy',
                                      optimizer='rmsprop',
                                      metrics=['accuracy']
                                          )
                            trial.setFit_Generator( 
                                            nb_epoch=epochs,
                                            callbacks=[earlyStopping, overfitStopping],
                                            max_q_size=100)
                                            #pickle_safe=True)



                            print("EXECUTE: ", name,labels, depth,width, activation_name)
                            trial.execute(custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                         train_arg_decode_func=label_dir_pairs_args_decoder,
                                         val_arg_decode_func=label_dir_pairs_args_decoder)


                            trial.test(test_proc=test,
                                         test_samples=num_test,
                                         custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                        arg_decode_func = label_dir_pairs_args_decoder)



                            trial.to_record({"lables": labels,
                                             "depth": depth,
                                             "width": width,
                                             "activation": activation_name,
                                             "dropout":dropout
                                            })
                            if(name == "lorentz"):
                                trial.to_record({"sphereCoords" : sphereCoords})
                                       

                
        

    

[(0, 88816), (88816, 19032), (107848, 19032)]
('EXECUTE: ', 'lorentz', ['ttbar', 'wjet'], 1, 10, 'relu')
Generating DataProcedure in range(88816,107848):

In [ ]:
histories = []
custom_objects ={'Lorentz': Lorentz, 'Slice': Slice}
for i, trial in enumerate(trials):
    #Execute and test
    test_proc = PreprocessingProcedure(archive_dir, True,preprocessFromPandas_label_dir_pairs,
                            ldps_pairs[i], num_samples*2,num_samples, object_profiles,observ_types)
    trial.execute(arg_decode_func=label_dir_pairs_args_decoder, custom_objects=custom_objects)
    metrics = trial.test(test_proc,custom_objects=custom_objects)
    print(metrics)
    
    #Graphs n such
    model = trial.get_model()
    #image_name = 'model_%r.png' % trial.hash()
    image_name = "model_" +str(i)+ ".png"
    dot = plot(model, to_file=image_name, show_shapes=True, show_layer_names=False)
    histories.append( (trial, Image(image_name), trial.get_history(), trial.get_from_index("val_acc") ) )

In [ ]:
for trial, image, histDict, val_acc in histories:
    display(image)
    history = History()
    history.history = histDict
    plot_history([("dense", history)])
    print(trial.hash() + ': Best Validation accuracy: %s' % "{:0.4f}".format(val_acc))
    
    

In [ ]:
import matplotlib.pyplot as plt
bar_width = 0.35
depths = [ trial.get_from_index("depth") for trial, image, histDict, val_acc in histories]
index = np.arange(len(depths))
accuracies = [val_acc for trial, image, histDict, val_acc in histories]

rects1 = plt.scatter(index, accuracies,
                 color='b',
                 label='Men')


plt.xlabel('Depth')
plt.ylabel('Validation Accuracy')
plt.title('Model validation accuracy by depth')
plt.xticks(index, depths)
#plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

#smartCheckpoint = SmartCheckpoint("dense"+add_title)
#RUN Dense
#dense_history = dense.fit(X_train_flatten, y_train,
#                    batch_size=batch_size,
#                    nb_epoch=epochs,
 #                   validation_split=.2,
#                    callbacks=[earlyStopping, overfitStopping])
#histories["dense"+add_title] = (dense,dense_history,X_train_flatten, y_train)

In [ ]:
keys = [key for key in histories]
def p(key):
    tup = histories[key]
    model = tup[0]
    history = tup[1]
    #print_accuracy_m(model, tup[2], tup[3])
    print(key + ': Best Validation accuracy: %r%%' % max(history.history["val_acc"]))
    plot_history([(key, history)])


In [ ]:
p(keys[0])

In [ ]:
p(keys[1])

In [ ]:
p(keys[2])

In [ ]:
p(keys[3])

# import json
class KerasStore:
    def __init__(self,
                    label_dir_pairs=None,
                    num_samples=None,
                    object_profiles=None,
                    observ_types=None,
                )
    def to_JSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
    
a = Object()
a.label_dir_pairs = label_dir_pairs
a.num_samples = num_samples
a.object_profiles = object_profiles
a.observ_types = observ_types


f = JSONDecoder(object_hook = from_json).decode('{"fname": "/foo/bar"}')
print("JSON:")
print(a.to_JSON())



def setPreprocessingInfo(label_dir_pairs, num_samples, object_profiles, observ_types):
    hashString = ""
    for x, y in label_dir_pairs:
        hashString += x
        hashString += y
    hashString += num_samples
    for x in object_profiles:
        hashString += str(x)
    for x in observ_types:
        hashString += x
    
   